In [30]:
import pandas as pd 
import numpy as np 


In [31]:
df1=pd.read_csv('Combined_Jobs_Final.csv')


In [32]:
df1['Title'] = df1['Title'] + ', City: ' + df1['City'] + ', State : ' +df1['State.Name'] 

In [33]:
df1 = df1.drop(['Created.At', 'Updated.At','Latitude', 'Longitude','Listing.Start','Listing.End','Employment.Type','Industry'], axis=1)


In [34]:
df1 = df1.drop(['State.Code','Company','Slug','Job.ID', 'Provider','Slug','Requirements','Address','Education.Required','Requirements','Status','Salary'], axis=1)

In [35]:
df1.columns

Index(['Title', 'Position', 'City', 'State.Name', 'Job.Description'], dtype='object')

In [36]:
df1.dropna(subset=['City'], inplace=True)

In [37]:
df1.dropna(subset=['State.Name'], inplace=True)

In [38]:
df1.shape

(83916, 5)

In [39]:
df1.head()

,Title,Position,City,State.Name,Job.Description
0,"Server @ Tacolicious, City: Palo Alto, State :...",Server,Palo Alto,California,Tacolicious' first Palo Alto store just opened...
1,"Kitchen Staff/Chef @ Claude Lane, City: San Fr...",Kitchen Staff/Chef,San Francisco,California,\r\n\r\nNew French Brasserie in S.F. Financia...
2,"Bartender @ Machka Restaurants Corp., City: Sa...",Bartender,San Francisco,California,We are a popular Mediterranean wine bar and re...
3,"Server @ Teriyaki House, City: Brisbane, State...",Server,Brisbane,California,● Serve food/drinks to customers in a profess...
4,"Kitchen Staff/Chef @ Rosa Mexicano - Sunset, C...",Kitchen Staff/Chef,Los Angeles,California,"Located at the heart of Hollywood, we are one ..."


Now we have only included the columns which have relavent information reggarading the Job Titles

In [40]:
df1=df1.sample(n=1000,random_state=42)

In [41]:
df=df1
df.head()


,Title,Position,City,State.Name,Job.Description
35567,Event Coordinator (Entry Level) @ Beyond Marke...,Event Coordinator (Entry Level),Columbus,Ohio,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nEvent Coor...
16167,Psychiatric Technician - PRN - Float @ St. Luk...,Psychiatric Technician - PRN - Float,Phoenix,Arizona,Provides direct patient care in the acute care...
37311,Center Managers and Service Associates @ Ace C...,Center Managers and Service Associates,Wilmington,Delaware,ACE Cash Express is the largest and fastest gr...
42797,Sales Assistant - Great Mid-Wilshire Firm! @ O...,Sales Assistant - Great Mid-Wilshire Firm!,Los Angeles,California,Ref ID: 00320-160417Classification: Secretary/...
2970,"Macy's Seasonal Retail Fitting Room Associate,...","Macy's Seasonal Retail Fitting Room Associate,...",Los Angeles,California,Job Overview:\r\n\r\nAs a Seasonal Fitting Roo...


In [42]:
df=df.reset_index()

In [43]:
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [44]:
def cleaning(txt):
    txt=re.sub(r'[^a-zA-Z0-9\s]',' ',txt)
#'/s' represents spaces,tabs and next lines 
    tokens=nltk.word_tokenize(txt.lower())
    stemming=[ps.stem(w) for w in tokens if w not in stopwords.words('english')]
    return " ".join(stemming)    


In [45]:
df['State.NameT']=df['State.Name']

In [46]:
df['Job.Description'] = df['Job.Description'].astype(str).apply(lambda x: cleaning(x))
df['Title2'] = df['Title'].astype(str).apply(lambda x: cleaning(x))

In [47]:
df['City'] = df['City'].astype(str).apply(lambda x: cleaning(x))
df['State.Name'] = df['State.Name'].astype(str).apply(lambda x: cleaning(x))

In [48]:
df['clean_text'] = df['Title2']+" "+df['Job.Description']+df['City']+df['State.Name']

In [80]:
df['State.NameT'].value_counts().head(15)

State.NameT
California        140
Florida            72
Pennsylvania       65
Ohio               53
Texas              50
Illinois           47
Washington         46
New York           38
Minnesota          35
New Jersey         31
Michigan           28
North Carolina     27
Massachusetts      27
Missouri           25
Wisconsin          24
Name: count, dtype: int64

<h2>Vectorization 

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
tfidf=TfidfVectorizer()
matrix=tfidf.fit_transform(df['clean_text'])
similarity = cosine_similarity(matrix)

In [58]:
df['Position'].value_counts().head(15)

Position
Administrative Assistant                           19
Customer Service Representative                    18
Staff Accountant                                   14
Accounting Clerk                                   10
Accounts Payable Clerk                              8
Receiving/Stock Associate                           8
Accounts Receivable Clerk                           7
Customer Service / Sales ( New Grads Welcome! )     7
Full Charge Bookkeeper                              6
Bookkeeper                                          6
Retail Sales Associate / Photographer               5
Senior Accountant                                   5
Retail Sales Associate – Part-Time                  5
Data Entry Clerk                                    5
Receptionist                                        5
Name: count, dtype: int64

In [59]:
df['PositionT']=df['Position']+" "+df['State.NameT']

In [79]:
df['PositionT'].value_counts().head(5)

PositionT
Customer Service Representative California                                       5
Full Charge Bookkeeper Washington                                                3
Staff Accountant Washington                                                      3
Mechanic Arizona                                                                 3
Retail Sales Associate - Showroom Sales - Interior Design Consultant Illinois    3
Name: count, dtype: int64

<h5>-'enumerate' function is used to create a tuple of index and similarity score 
Eg:(0,1),(1,0.533),(2,0.78)....(n,score)
<h5>- Now we use key=lambda x:x[1] because the x[0] is index and  x[1] similarity score so we arrange the Recommendations w.r.t similarity score

In [81]:
def recommend(Position):

    indx = df[df['PositionT'] == Position].index[0]
    distances = sorted(list(enumerate(similarity[indx])), key=lambda x: x[1], reverse=True)[1:10]
    # Now we have the indexes of common items . So we can decide what we want for Recommendations
    jobs = []
    for i in distances:
        jobs.append(df.iloc[i[0]]['Title'])
    return jobs

# Example usage
recommended_jobs = recommend('Server California')
print(recommended_jobs)

['Marketplace Server @ Presbyterian Senior Living, City: Bethlehem, State : Pennsylvania', 'Operations Specialist Opportunity in San Francisco @ Accountemps, City: San Francisco, State : California', 'Seeking Recent Graduates! @ OfficeTeam, City: San Francisco, State : California', 'Bartender @ American Golf, City: Thousand Oaks, State : California', 'Dietary Aide - Wesley Acres - Des Moines @ WesleyLife, City: Des Moines, State : Iowa', 'RECENT GRADS APPLY TODAY! @ OfficeTeam, City: San Francisco, State : California', 'Dishwashers, Prep Cooks, Grill Cooks, Lead Cooks, Line Servers, Cashiers, Concessions, Banquet Servers and Housekeepers @ All Team Staffing, City: Murfreesboro, State : Tennessee', 'Loan Associate Opportunity in San Francisco @ Accountemps, City: San Francisco, State : California', 'Dining Services Associates @ Regency Park and HarborChase of Vero Beach, City: Vero Beach, State : Florida']


-Now here by 'x=df.iloc[i[0]]' gives us an index value because i is (index,similrity_score)

-and by x['Title] we will get the value of title of that index

In [72]:
import pickle
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [73]:
import pandas as pd
print(pd.__version__)

2.0.0
